In [1]:
import pandas as pd

In [2]:
numeric_columns = [
 'requester_account_age_in_days_at_request',
 'requester_days_since_first_post_on_raop_at_request',
 'requester_number_of_comments_at_request',
 'requester_number_of_comments_in_raop_at_request',
 'requester_number_of_posts_at_request',
 'requester_number_of_posts_on_raop_at_request',
 'requester_number_of_subreddits_at_request',
 'requester_upvotes_minus_downvotes_at_request',
 'requester_upvotes_plus_downvotes_at_request',
 'unix_timestamp_of_request',
 'unix_timestamp_of_request_utc',
 'requester_received_pizza']

def read_data(mode):
    # WARNING: THIS FUNCTION IS DROPPING SPARSE COLUMNS
#     dataframe = pd.DataFrame()
#     dataframe.columns=numeric_columns
    if mode == 'train':
        dataframe = pd.read_json('./data/train.json')
        dataframe['requester_received_pizza'] = dataframe['requester_received_pizza'].astype(int)
    elif mode == 'test':
        dataframe = pd.read_json('./data/test.json')
        
    return dataframe
    

# def x_y_split(dataframe):
#     """ Splits a pd dataframe into the features and results """
#     features = dataframe.drop('requester_received_pizza', axis=1)
#     results = dataframe['requester_received_pizza']
#     return features, results
    

In [3]:
train_df = read_data('train')
train_df[numeric_columns].to_csv('./data/train.csv', index=False)
# test_df = read_data('test')
# test_df[numeric_columns[:-1]].to_csv('./data/test.csv', index=False)

In [4]:
# test_df[numeric_columns].to_csv('./data/train.csv')
#list(train_df.columns.intersection(test_df.columns))

In [5]:
#list(pd.read_csv('./data/train.csv').columns)

In [6]:
import tensorflow as tf

In [7]:
record_defaults = [tf.constant([], dtype=tf.float32) for c in range(len(numeric_columns))]
record_defaults

[<tf.Tensor 'Const:0' shape=(0,) dtype=float32>,
 <tf.Tensor 'Const_1:0' shape=(0,) dtype=float32>,
 <tf.Tensor 'Const_2:0' shape=(0,) dtype=float32>,
 <tf.Tensor 'Const_3:0' shape=(0,) dtype=float32>,
 <tf.Tensor 'Const_4:0' shape=(0,) dtype=float32>,
 <tf.Tensor 'Const_5:0' shape=(0,) dtype=float32>,
 <tf.Tensor 'Const_6:0' shape=(0,) dtype=float32>,
 <tf.Tensor 'Const_7:0' shape=(0,) dtype=float32>,
 <tf.Tensor 'Const_8:0' shape=(0,) dtype=float32>,
 <tf.Tensor 'Const_9:0' shape=(0,) dtype=float32>,
 <tf.Tensor 'Const_10:0' shape=(0,) dtype=float32>,
 <tf.Tensor 'Const_11:0' shape=(0,) dtype=float32>]

In [8]:
def input_fn(data_file, num_epochs, shuffle, batch_size):
    """ Generate an input function for the Estimator """
    assert tf.gfile.Exists(data_file), (f'{data_file} not found. Please make sure the data file is in the correct location.')
    
    def parse_csv(value):
        print('Parsing', data_file)
        columns = tf.decode_csv(value, record_defaults=record_defaults)
        features = dict(zip(numeric_columns, columns))
        labels = features.pop('requester_received_pizza')
        return features, labels
    
    dataset = tf.contrib.data.TextLineDataset(data_file) # in TF 1.4 data got moved to tf.contrib: https://github.com/tensorflow/tensorflow/releases/tag/v1.4.0
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=10)
        
    dataset = dataset.map(parse_csv, num_threads=5)
    
    dataset = dataset.repeat(num_epochs)
    dataset = dataset.batch(batch_size)
    
    iterator = dataset.make_one_shot_iterator()
    features, labels = iterator.get_next()
    return features, labels


In [9]:
# categorical columns
# post_was_edited true or false
# requester_received_pizza true or fasle
# requester_user_flair None, "schroom", "PIF"
# everything else numerical

tf_feature_columns = [tf.feature_column.numeric_column(x) for x in numeric_columns[:-1]]
# tensorflow features cannot accept strings. must transform strings to categorical or numerical

In [10]:
model_dir = './models/'
model = tf.estimator.LinearClassifier(
    model_dir=model_dir, 
    feature_columns=tf_feature_columns
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [12]:
train_data = './data/train.csv'
num_epochs = 1
shuffle = True
batch_size = 30
model.train(input_fn=lambda: input_fn(train_data, num_epochs, True, batch_size))


Parsing ./data/train.csv
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./models/model.ckpt.
INFO:tensorflow:loss = 20.7944, step = 1
INFO:tensorflow:global_step/sec: 614.743
INFO:tensorflow:loss = 3.6438e+08, step = 101 (0.163 sec)
INFO:tensorflow:Saving checkpoints for 135 into ./models/model.ckpt.
INFO:tensorflow:Loss for final step: 1.02013e+08.
